<a href="https://colab.research.google.com/github/shweta-todgire/Data-Science-Codes/blob/main/Handling_Categorical_Data_%26_Classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving StudentsPerformance.csv to StudentsPerformance.csv


In [20]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

df = pd.read_csv("/content/StudentsPerformance.csv")
feature = df[['lunch']]
ohe = OneHotEncoder(sparse_output=False)

encoded = ohe.fit_transform(feature)

encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(['lunch']))

print(encoded_df.head())

# Decode back to original labels
decoded = ohe.inverse_transform(encoded)
print(decoded[:5])


   lunch_free/reduced  lunch_standard
0                 0.0             1.0
1                 0.0             1.0
2                 0.0             1.0
3                 1.0             0.0
4                 0.0             1.0
[['standard']
 ['standard']
 ['standard']
 ['free/reduced']
 ['standard']]


In [3]:
pd.get_dummies(df['gender'])

,female,male
0,True,False
1,True,False
2,True,False
3,False,True
4,False,True
...,...,...
995,True,False
996,False,True
997,True,False
998,True,False


In [4]:
from sklearn.preprocessing import MultiLabelBinarizer

subjects = [
    ("math", "reading"),
    ("reading", "writing"),
    ("math", "writing"),
    ("reading", "writing"),
    ("math", "reading")
]

mlb = MultiLabelBinarizer()
encoded = mlb.fit_transform(subjects)

print(encoded)
print(mlb.classes_)


[[1 1 0]
 [0 1 1]
 [1 0 1]
 [0 1 1]
 [1 1 0]]
['math' 'reading' 'writing']


In [6]:
df['parental level of education']

scale_map = {
    "some high school": 1,
    "high school": 2,
    "some college": 3,
    "associate's degree": 4,
    "bachelor's degree": 5,
    "master's degree": 6
}

df['edu_encoded'] = df['parental level of education'].map(scale_map)

print(df[['parental level of education', 'edu_encoded']].head())



  parental level of education  edu_encoded
0           bachelor's degree            5
1                some college            3
2             master's degree            6
3          associate's degree            4
4                some college            3


In [21]:
from sklearn.feature_extraction import DictVectorizer

dict_data = df[["gender","race/ethnicity","lunch"]].to_dict(orient="records")

dv = DictVectorizer(sparse=False)
dict_features = dv.fit_transform(dict_data)

print(dict_features[:5])
print(dv.get_feature_names_out())


[[1. 0. 0. 1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 1. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 1. 0. 0.]]
['gender=female' 'gender=male' 'lunch=free/reduced' 'lunch=standard'
 'race/ethnicity=group A' 'race/ethnicity=group B'
 'race/ethnicity=group C' 'race/ethnicity=group D'
 'race/ethnicity=group E']


In [22]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# create missing values
data2 = df.copy()
data2.loc[0:3, "gender"] = np.nan

# Convert gender to numerical for prediction
gender_map = {"female":0, "male":1}
reverse_gender = {0:"female", 1:"male"}
data2["gender_num"] = data2["gender"].replace(gender_map)

# Features used to predict gender
X = data2[["reading score", "writing score"]].values
y = data2["gender_num"].values

# Train only on non-missing
mask = ~np.isnan(y)
knn = KNeighborsClassifier(3)
knn.fit(X[mask], y[mask])

# Predict missing
y_pred = knn.predict(X[~mask])
data2.loc[~mask, "gender_num"] = y_pred
data2["gender_imputed"] = data2["gender_num"].replace(reverse_gender)

data2.head(10)

/tmp/ipython-input-2656612793.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data2["gender_num"] = data2["gender"].replace(gender_map)


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,gender_num,gender_imputed
0,NaN,group B,bachelor's degree,standard,none,72,72,74,0.0,female
1,NaN,group C,some college,standard,completed,69,90,88,0.0,female
2,NaN,group B,master's degree,standard,none,90,95,93,0.0,female
3,NaN,group A,associate's degree,free/reduced,none,47,57,44,1.0,male
4,male,group C,some college,standard,none,76,78,75,1.0,male
5,female,group B,associate's degree,standard,none,71,83,78,0.0,female
6,female,group B,some college,standard,completed,88,95,92,0.0,female
7,male,group B,some college,free/reduced,none,40,43,39,1.0,male
8,male,group D,high school,free/reduced,completed,64,64,67,1.0,male
9,female,group B,high school,free/reduced,none,38,60,50,0.0,female


In [29]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

data2 = df.copy()
data2.loc[0:3, "gender"] = np.nan

gender_map = {"female": 0, "male": 1}
reverse_gender = {0: "female", 1: "male"}
data2["gender_num"] = data2["gender"].map(gender_map)

X = data2[["reading score", "writing score"]].values
y = data2["gender_num"].values

mask = ~np.isnan(y)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X[mask], y[mask])

y_pred = knn.predict(X[~mask])
data2.loc[~mask, "gender_num"] = y_pred

data2["gender_imputed"] = data2["gender_num"].map(reverse_gender)

data2.head(10)


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,gender_num,gender_imputed
0,NaN,group B,bachelor's degree,standard,none,72,72,74,0.0,female
1,NaN,group C,some college,standard,completed,69,90,88,0.0,female
2,NaN,group B,master's degree,standard,none,90,95,93,0.0,female
3,NaN,group A,associate's degree,free/reduced,none,47,57,44,1.0,male
4,male,group C,some college,standard,none,76,78,75,1.0,male
5,female,group B,associate's degree,standard,none,71,83,78,0.0,female
6,female,group B,some college,standard,completed,88,95,92,0.0,female
7,male,group B,some college,free/reduced,none,40,43,39,1.0,male
8,male,group D,high school,free/reduced,completed,64,64,67,1.0,male
9,female,group B,high school,free/reduced,none,38,60,50,0.0,female


In [32]:
df["pass_math"] = (df["math score"] >= 60).astype(int)

target = df["pass_math"].values
features = df[["reading score","writing score"]].values


In [33]:
import numpy as np
unique, counts = np.unique(target, return_counts=True)
print(dict(zip(unique, counts)))


{np.int64(0): np.int64(323), np.int64(1): np.int64(677)}


In [38]:
from sklearn.ensemble import RandomForestClassifier

# Convert target: completed=1, none=0
df['test_prep_binary'] = np.where(df['test preparation course'] == 'completed', 1, 0)

X = df[['math score', 'reading score', 'writing score']]
y = df['test_prep_binary']

# Check imbalance
y.value_counts()

,count
test_prep_binary,
0,642
1,358


In [39]:
clf = RandomForestClassifier(class_weight='balanced', random_state=42)
clf.fit(X, y)


RandomForestClassifier(class_weight='balanced', random_state=42)

In [40]:
weights = {0: 0.7, 1: 0.3}
clf_weighted = RandomForestClassifier(class_weight=weights, random_state=42)
clf_weighted.fit(X, y)


RandomForestClassifier(class_weight={0: 0.7, 1: 0.3}, random_state=42)

In [41]:
import numpy as np

i_class0 = np.where(y == 0)[0]
i_class1 = np.where(y == 1)[0]

# Upsample minority
i_class1_upsampled = np.random.choice(i_class1, size=len(i_class0), replace=True)

# Combine
X_upsampled = np.vstack((X.iloc[i_class0], X.iloc[i_class1_upsampled]))
y_upsampled = np.hstack((y.iloc[i_class0], y.iloc[i_class1_upsampled]))

pd.Series(y_upsampled).value_counts()


,count
0,642
1,642
